# Priorização de Tasks

**Objetivo:** Desenvolver um método para gerar um score que meça o provável sucesso de uma Task manual.
***

**O que é uma Task?**  
Trata-se de uma atividade voltador para os vendedores (SDRs e Closers) executarem e cujo foco é conseguir uma resposta do lead prospectado.
***

**Quais os tipos de Task?**
 - Envio de email
 - Contato via linkedin
 - Telefonema
 - Contato via alguma outra mídia social
 - Reunião
***

**O que é uma Task de sucesso? Em ordem de prioridade:**
 1. Uma task que tenha tido uma resposta positiva
 2. Uma task que tenha tido uma resposta, mesmo que negativa
 3. Demais tasks sem resposta
***

**Quais fatores podem influenciar o sucesso de uma task?**
 - Contatos que já tiveram uma interação prévia tem mais chances de sucesso? 
 - Contatos com engajamento recente devem ser priorizados?
 - Ozzy Tasks realmente tem mais chances de sucesso do que tasks tradicionais? 
 - Existem templates de email que favoreçam a resposta do lead?
 - Existe alguma correlação de data/hora e taxa de sucesso? (considerar a tabela de flow_actions para essa análise)
 - Correlação entre tipos de tarefas (email/linkedin/social/telefonema) e taxa de sucesso. 
 - Correlação entre as interações do contato (interações com emails, ligações, etc) e a taxa de sucesso.
 - Correlação entre tempo de resposta e taxa de sucesso.
 - Correlações entre estágios e taxa de sucesso. 
 - Origem (contatos que chegam via Inbound/LeadGen).
 - Conversões de Inbound (RD e Hubspot).
 - O perfil do lead influencia?
 - O perfil da empresa prospectante influencia?
***

**Observações:**
 - A task do tipo reunião fica fora do nosso escopo de análise por ser conceitualmente diferente das demais. Para que exista uma reunião o lead já tem que ter acenado positivamente e portanto não há necessidade criar um score para a reunião. Reuniões sempre terão prioridade em seu horário marcado.
 - É preciso tomar cuidado para que não haja tasks que nunca sejam elencadas para execução.
 

# Quais os tipos de Task?

In [ ]:
import os
import utils
import pandas as pd

query = '''select distinct(service) from tasks where type = 'ManualTask' order by service'''
df = pd.read_sql(query, os.environ['REEVAPI_URL'])

df

# O que é uma Task de sucesso? 

**Table:** `tasks join events on tasks.id = events.task_id`

- O objetivo dessa análise é criar um atributo target Sucesso com valores True/False
- É esperado um método que leia o status de cada tipo de Task e consiga classificá-lo em True/False

### Quais tipos de eventos são necessários para nossa análise?

In [ ]:
import os
import utils
import pandas as pd

query = '''
            select 
                distinct(events.type) 
            from 
                events
        '''

df = pd.read_sql(query, os.environ['REEVAPI_URL'])

df

O evento `EmailEvent` é o mais descrito para o trabalho a ser feito.

`CallEvent` é gerado quando a ligação é feita internamente via voip, no entanto pode haver ligações feitas através de telefones particulares que podem ter um `ManualTaskEvent` associado.

Um `ManualTaskEvent` é gerado para cada etapa de uma tarefa manual realizada. Em especial é possível tirar informações para Linkedin, Social e Ligações (feitas por fora do Reev).

### Recupera exemplo (id) de Tasks de envio de email com Reply

In [ ]:
import os
import utils
import pandas as pd

query = '''
            select 
                *
            from 
                tasks
            where 
                type = 'ManualTask'
            and 
                service = 'ManualTasksServices::Email'
            limit 
                20
        '''

df = pd.read_sql(query, os.environ['REEVAPI_URL'])

df

### Utiliza a task de exemplo para entender o fluxo de acontecimentos

In [ ]:
import os
import utils
import pandas as pd

query = '''
            select 
                *
            from 
                events
            where 
                task_id is not null
            limit 100
        '''

df = pd.read_sql(query, os.environ['REEVAPI_URL'])

df

### Checando Flow_Recipients

In [ ]:
import os
import utils
import pandas as pd

query = '''
            select 
                count(fr.contact_final_stage), 
                fr.contact_final_stage
            from 
                flow_recipients fr
            join
                tasks t
            on fr.contact_id = t.referenceable_id
            where
                t.referenceable_type = 'Contact'
            and
                fr.contact_final_stage is not null
            group by
                fr.contact_final_stage
        '''

df = pd.read_sql(query, os.environ['REEVAPI_URL'])

df

In [ ]:
import os
import utils
import pandas as pd

query = '''
            select 
                t.created_at as t_created_at,
                t.due_on as t_due_on,
                t.done_at as t_done_at,
                t.service as t_service,
                t.type as t_type,
                fr.flow_id as fr_flow_id,
                fr.last_flow_action_taken as fr_last_flow_action_taken,
                case when fr.contact_final_stage = 'lost' then 0 else 1 end as target
            from 
                flow_recipients fr
            join
                tasks t
            on fr.contact_id = t.referenceable_id
            where
                t.referenceable_type = 'Contact'
            and
                t.done_at is not null
            and
                fr.contact_final_stage is not null
            limit
                20
        '''

df = pd.read_sql(query, os.environ['REEVAPI_URL'])

df

In [ ]:
#datas antigas, mesma query

### Montando um dataframe de flow_recipients com balanceamento de classe

In [ ]:
import os
import utils
import pandas as pd

stages = ['client', 'lead', 'lost', 'mql', 'opportunity', 'prospect', 'smart_lead', 'sql']
dfs = []

for s in stages:
    query = '''
            select 
                t.created_at as t_created_at,
                t.due_on as t_due_on,
                t.done_at as t_done_at,
                t.service as t_service,
                t.type as t_type,
                fr.flow_id as fr_flow_id,
                fr.last_flow_action_taken as fr_last_flow_action_taken,
                case when fr.contact_final_stage = 'lost' then 0 else 1 end as target
            from 
                flow_recipients fr
            join
                tasks t
            on fr.contact_id = t.referenceable_id
            where
                t.referenceable_type = 'Contact'
            and
                t.done_at is not null
            and
                fr.contact_final_stage = '{}'
            limit
                1000
        '''.format(s)

    df = pd.read_sql(query, os.environ['REEVAPI_URL'])

    dfs.append(df)

In [ ]:
df

In [ ]:
df = pd.concat(dfs)

df.shape
    

# Contatos que já tiveram uma interação prévia tem mais chances de sucesso?

**Table:** `contacts`  

Essa tabela guarda os dados de leads que estão sendo prospectados. É importante notar que um mesmo lead pode ser prospectado mais de uma vez por uma mesma empresa e até mesmo por diferentes empresas.

** Pontos a considerar**
- considerar tarefas automáticas para análise total
- porcentagem de contatos que são prospectados por mais de uma empresa
- distribuição do número de etapas realizadas com contatos


In [ ]:
import os
import utils
import pandas as pd

query = '''select email from contacts'''
df = pd.read_sql(query, os.environ['REEVAPI_URL'])

utils.save_pkl('df_contacts.emails', df)

In [ ]:
df = utils.open_pkl('df_contacts.emails')
df

In [ ]:
print('Número total de leads:',df.email.count())
print('Número de leads únicos:',len(df.email.unique()))

In [ ]:
emails = dict()

for i in range(0, df.shape[0]):
    if df.email[i] not in emails: emails[df.email[i]] = 0
    emails[df.email[i]] += 1

In [ ]:
emails2 = dict()

for email in emails:
    if emails[email] not in emails2: emails2[emails[email]] = 0
    emails2[emails[email]] += 1

In [ ]:
utils.cdf([emails2], x_label='Frequência', y_label='% E-mails')

In [ ]:
import operator

sorted_emails = sorted(emails.items(), key=operator.itemgetter(1), reverse=True)
    
sorted_emails

In [ ]:
utils.cdf([emails2], x_label='Frequência', y_label='% E-mails', xleft=1, xright=10)

# OzzyTasks possuem mais chance de sucesso?
Existem dois tipos de ozzy tasks:   
  1. ozzy tasks de contatos que estejam em um fluxo
  2. ozzy tasks de contatos que não estejam em um fluxo (ex: contatos já perdidos)   
 
### Recuperando todas as ozzy tasks

In [ ]:
import os
import pandas as pd

query = '''select 
            e.eventable_id as contact_id,
            e.task_id,
            e.eventable_type,
            t.done_at
           from events e join tasks t on e.task_id = t.id 
           where (t.metadata->'ozzy')::boolean is True
            and t.type = 'ManualTask'
           '''
df = pd.read_sql(query, os.environ['REEVAPI_URL'])
df['done_at'] = pd.to_datetime(df['done_at'], errors='coerce')

Criando um conjunto apenas com os ids de contatos que tiveram uma ozzy task associada

In [ ]:
contacts = set()

for i in range(0, df.shape[0]):
    contacts.add(int(df.contact_id[i]))

Criando uma lista de contatos e datas de execução da ozzy task associada

In [ ]:
contacts_dates_tasks = []

for i in range(0, df.shape[0]):
    contacts_dates_tasks.append((int(df.contact_id[i]), df.done_at[i], df.task_id))

### Recuperando as ozzy tasks do tipo 1 (fluxo associado)   
Recupera todos os flow_recipients dos contatos para posteriormente filtrar os que se enquandram na data de execução da ozzy task:   
`fr.created_at < t.done_at < fr.status_updated_at`

In [ ]:
import os
import pandas as pd

query = '''select 
            fr.contact_id,
            fr.status,
            fr.created_at,
            fr.status_updated_at,
            fr.contact_final_stage,
            count(fa.flow_id) as fa_steps,
            (fr.last_flow_action_taken + 1) as fr_last_flow_action_taken
           from flow_recipients fr
           join flow_actions fa on fr.flow_id = fa.flow_id
           where
            fr.contact_id in (**{}**)
           group by
            fr.contact_id,
            fr.status,  
            fr.created_at,
            fr.status_updated_at,
            fr.contact_final_stage,
            fr.last_flow_action_taken
           '''.format(str(contacts))

query = query.replace('**{', '').replace('}**', '')
df2 = pd.read_sql(query, os.environ['REEVAPI_URL'])
df2['created_at'] = pd.to_datetime(df2['created_at'], errors='coerce')
df2['status_updated_at'] = pd.to_datetime(df2['status_updated_at'], errors='coerce')

In [ ]:
df2

In [ ]:
# removendo NaNs e tratando valores float
df2.fr_last_flow_action_taken = df2.fr_last_flow_action_taken.fillna(0)
df2.fr_last_flow_action_taken = df2.fr_last_flow_action_taken.astype(int)

In [ ]:
# filtrando pela data/hora de execução da ozzy task
import datetime

dfs = []

for i in contacts_dates_tasks:
    temp = df2[(df2['contact_id'] == i[0]) & (df2['created_at'] < i[1]) & (df2['status_updated_at'] > i[1])]
    dfs.append(temp)
    
df3 = pd.concat(dfs)
df3 = df2.reset_index(drop=True)

In [ ]:
df3

Adicionando o target   

**Importante notar que há muitos flow_recipients que não se enquadram em nenhum dos targets**

In [ ]:
def get_target(status, contact_final_stage, fa_steps, fr_last_flow_action_taken):
    # connected + client
    if status == 'connected' and contact_final_stage == 'client':
        return 'connected_client'

    # connected
    if status == 'connected' and contact_final_stage != 'client':
        return  'connected'
        
    # converting
    if status == 'converting':
        return 'converting'
        
    # lost + finished e parou no meio do fluxo
    if contact_final_stage == 'lost' and fa_steps != fr_last_flow_action_taken:
        return 'lost'

    # lost + finished e passou por todo o fluxo
    if contact_final_stage == 'lost' and fa_steps == fr_last_flow_action_taken:
        return 'lost_whole_flow'

    
df3['target']= df3.apply(lambda x: get_target(x['status'],x['contact_final_stage'],x['fa_steps'],x['fr_last_flow_action_taken']),axis=1)

In [ ]:
# tasks que não foram ser associadas a um target
df_result = df3[df3.target.notnull()]

In [ ]:
df_result

### Ozzies Tasks type 2

# Calcula a porcentagem de conversão de ozzy tasks vs task manuais

In [ ]:
import os
import utils
import pandas as pd

dfs = []

query = '''
        select
            count(*)
        from 
            flow_recipients fr join tasks t on fr.contact_id = t.referenceable_id 
            join contacts c on fr.contact_id = c.id 
        (where)
    '''

# connected + client
target = '''
            'connected_client' as target
        '''

where = '''
        where
            fr.last_flow_action_taken is not null
            and (fr.status = 'finished' or fr.status = 'connected')
            and fr.contact_final_stage = 'client' 
            and t.referenceable_type = 'Contact' 
            and t.done_at is not null
            and t.type = 'ManualTask'
        '''

query_to_execute = query.replace('(target)', target).replace('(where)', where)
df = pd.read_sql(query_to_execute, os.environ['REEVAPI_URL'])

connected_client = df['count'][0]

# connected
target = '''
            'connected' as target
        '''

where = '''
        where
            fr.last_flow_action_taken is not null
            and fr.status = 'connected' 
            and fr.contact_final_stage <> 'client' 
            and t.referenceable_type = 'Contact' 
            and t.done_at is not null
            and t.type = 'ManualTask'
       '''

query_to_execute = query.replace('(target)', target).replace('(where)', where)
df = pd.read_sql(query_to_execute, os.environ['REEVAPI_URL'])

connected = df['count'][0]

# converting
target = '''
            'converting' as target
        '''

where = '''
        where
            fr.last_flow_action_taken is not null
            and fr.status = 'converting'  
            and t.referenceable_type = 'Contact' 
            and t.done_at is not null
            and t.type = 'ManualTask'
    '''

query_to_execute = query.replace('(target)', target).replace('(where)', where)
df = pd.read_sql(query_to_execute, os.environ['REEVAPI_URL'])

converting = df['count'][0]

# lost + finished e parou no meio do fluxo
target = '''
            'lost' as target
        '''

where = '''
        where
            fr.last_flow_action_taken is not null
            and fr.status = 'finished' 
            and fr.contact_final_stage = 'lost' 
            and t.referenceable_type = 'Contact' 
            and t.done_at is not null
            and t.type = 'ManualTask'
        '''

query_to_execute = query.replace('(target)', target).replace('(where)', where)
df = pd.read_sql(query_to_execute, os.environ['REEVAPI_URL'])

lost = df['count'][0]

total = pd.read_sql('select count(*) from tasks', os.environ['REEVAPI_URL'])['count'][0]

In [ ]:
total

In [ ]:
print(connected_client/total)
print(connected/total)
print(converting/total)
print(lost/total)

In [ ]:
ozzy_total = df3.shape[0]

ozzy_connected_client = df3[df3.target == 'connected_client'].shape[0]
ozzy_connected = df3[df3.target == 'connected'].shape[0]
ozzy_converting = df3[df3.target == 'converting'].shape[0]
ozzy_lost = df3[df3.target == 'lost'].shape[0] + df3[df3.target == 'lost_whole_flow'].shape[0]

In [ ]:
print(ozzy_connected_client/ozzy_total)
print(ozzy_connected/ozzy_total)
print(ozzy_converting/ozzy_total)
print(ozzy_lost/ozzy_total)

In [ ]:
print((ozzy_connected/ozzy_total)/(connected/total))

### Contatos que tenham interagido com alguma ozzy task, tem mais chance de se conectarem?
recupera uma amostragem de tasks

In [ ]:
import os
import utils
import pandas as pd

def get_dataframe():

    limit = 50000

    dfs = []
    
    query = '''
            select
                c.email,
                c.id as contact_id,
                t.company_id as t_company_id,
                t.metadata,
                t.type as t_type,
                t.done_at as t_done_at,
                t.service as t_service,
                fr.status as fr_status,
                fr.contact_id as fr_contact_id,
                fr.flow_id as fr_flow_id,
                count(fa.flow_id) as fa_steps,
                (fr.last_flow_action_taken + 1) as fr_last_flow_action_taken,
                (target)
            from 
                flow_recipients fr join tasks t on fr.contact_id = t.referenceable_id 
                join flow_actions fa on fr.flow_id = fa.flow_id
                join contacts c on fr.contact_id = c.id 
            (where)
            group by
                t.metadata,
                c.id,
                t.company_id,
                fr.contact_id,
                c.email,
                fa.flow_id, 
                t.type,
                t.done_at, 
                t.service, 
                t.type, 
                fr.status, 
                fr.flow_id, 
                fr.last_flow_action_taken
            (having)
            limit
                {}
        '''.format(limit)
    
    # connected + client
    target = '''
                'connected_client' as target
            '''
    
    where = '''
            where
                fr.last_flow_action_taken is not null
                and (fr.status = 'finished' or fr.status = 'connected')
                and fr.contact_final_stage = 'client' 
                and t.referenceable_type = 'Contact' 
                and t.done_at is not null
                and t.type = 'ManualTask'
            '''
    
    query_to_execute = query.replace('(target)', target).replace('(where)', where).replace('(having)', '')
    df = pd.read_sql(query_to_execute, os.environ['REEVAPI_URL'])
    dfs.append(df)
    
    # connected
    target = '''
                'connected' as target
            '''
    
    where = '''
            where
                fr.last_flow_action_taken is not null
                and fr.status = 'connected' 
                and fr.contact_final_stage <> 'client' 
                and t.referenceable_type = 'Contact' 
                and t.done_at is not null
                and t.type = 'ManualTask'
           '''
    
    query_to_execute = query.replace('(target)', target).replace('(where)', where).replace('(having)', '')
    df = pd.read_sql(query_to_execute, os.environ['REEVAPI_URL'])
    dfs.append(df)

    # converting
    target = '''
                'converting' as target
            '''
    
    where = '''
            where
                fr.last_flow_action_taken is not null
                and fr.status = 'converting'  
                and t.referenceable_type = 'Contact' 
                and t.done_at is not null
                and t.type = 'ManualTask'
        '''
    
    query_to_execute = query.replace('(target)', target).replace('(where)', where).replace('(having)', '')
    df = pd.read_sql(query_to_execute, os.environ['REEVAPI_URL'])
    dfs.append(df)

    # lost + finished e parou no meio do fluxo
    target = '''
                'lost' as target
            '''
    
    where = '''
            where
                fr.last_flow_action_taken is not null
                and fr.status = 'finished' 
                and fr.contact_final_stage = 'lost' 
                and t.referenceable_type = 'Contact' 
                and t.done_at is not null
                and t.type = 'ManualTask'
            '''

    having = '''
                having count(fa.flow_id) <> (fr.last_flow_action_taken + 1)
            '''
    
    query_to_execute = query.replace('(target)', target).replace('(where)', where).replace('(having)', having)
    df = pd.read_sql(query_to_execute, os.environ['REEVAPI_URL'])
    dfs.append(df)

    # lost + finished e passou por todo o fluxo
    target = '''
                'lost_whole_flow' as target
            '''
    
    where = '''
            where
                fr.last_flow_action_taken is not null
                and fr.status = 'finished' 
                and fr.contact_final_stage = 'lost' 
                and t.referenceable_type = 'Contact' 
                and t.done_at is not null
                and t.type = 'ManualTask'
            '''

    having = '''
                having count(fa.flow_id) = (fr.last_flow_action_taken + 1)
            '''
            
    query_to_execute = query.replace('(target)', target).replace('(where)', where).replace('(having)', having)    
    df = pd.read_sql(query_to_execute, os.environ['REEVAPI_URL'])
    dfs.append(df)

    df = pd.concat(dfs)

    return df

In [ ]:
df = get_dataframe()
df

recupera a lista de todos os contatos/empresas que tiveram intereção com uma ozzy task

In [ ]:
query = '''select 
            e.eventable_id as contact_id,
            t.company_id
           from events e join tasks t on e.task_id = t.id 
           where (t.metadata->'ozzy')::boolean is True
            and t.type = 'ManualTask'
           '''
df2 = pd.read_sql(query, os.environ['REEVAPI_URL'])

In [ ]:
df2

In [ ]:
contact_company = set()
for i in range(0, df2.shape[0]):
    c1 = df2.contact_id[i]
    c2 = df2.company_id[i]
    c3 = str(c1) + '_' + str(c2)
    contact_company.add(c3)

In [ ]:
df['ozzy'] = 0
df = df.reset_index(drop=True)

In [ ]:
def have_ozzy(contact, company):
    c = str(contact) + '_' + str(company)
    if c in contact_company:
        return 1
    return 0

df['ozzy'] = df.apply(lambda x: have_ozzy(x['contact_id'], x['t_company_id']), axis = 1)  

In [ ]:
df['ozzy'].unique()

In [ ]:
df_ozzy = df[df['ozzy'] == 1]
df_not_ozzy = df[df['ozzy'] == 0]

In [ ]:
ozzy_total = df_ozzy.shape[0]
ozzy_connected = df_ozzy[df_ozzy['target'] == 'connected_client'].shape[0] + df_ozzy[df_ozzy['target'] == 'connected'].shape[0]

o = ozzy_connected/ozzy_total
o

In [ ]:
not_ozzy_total = df_not_ozzy.shape[0]
not_ozzy_connected = df_not_ozzy[df_not_ozzy['target'] == 'connected_client'].shape[0] + df_not_ozzy[df_not_ozzy['target'] == 'connected'].shape[0]

n = not_ozzy_connected/not_ozzy_total
n

In [ ]:
o/n